## An introduction to the DataFrames.jl package and GLM

### The packages used in the tutorial

installed packages

In [ ]:
using DataFrames        # core package
using CSV               # working with CSV files
using FreqTables        # frequency tables
using Plots, StatsPlots # plotting
using GLM               # regression models
using Distributions     # statistical distributions
using CategoricalArrays # handling categorical data

standard library

In [ ]:
using Statistics

### Reading in a basic CSV file

We will work with data described and provided [here](https://archive.ics.uci.edu/ml/datasets/bank+marketing).

Our goal is to predict if the client will subscribe a term deposit (variable `"y"`).

In [ ]:
bank = CSV.read("bank-additional-full.csv", DataFrame)

Some useful facts are that when displaying a data frame you get an information about its size on top and element types contained in its columns just under column names.

The CSV.jl package does autodetection of delimiters. Let us inspect the original file visually:

In [ ]:
readlines("bank-additional-full.csv")

### Your first steps with `DataFrame`

Jou can get a list of names in a data frame as strings using the `names` function.

In [ ]:
names(bank)

Let us extract some columns:

In [ ]:
bank.age

In [ ]:
bank[:, "age"]

What is column name is non standard?

In [ ]:
bank.cons.conf.idx

In [ ]:
bank."cons.conf.idx"

In [ ]:
bank[:, "cons.conf.idx"]

Our data frame looks a bit wide. Let us keep only relevant cokumns in it.

In [ ]:
bank2 = select(bank, "y", "age", "campaign", "pdays")

In [ ]:
bank2 = select(bank, ["y", "age", "campaign", "pdays"])

You can also use rules to select columns. For example, pick the columns that start with `"cons"`

In [ ]:
select(bank, Cols(startswith("cons")))

In [ ]:
select(bank, r"^cons", "pdays")

Available column selectors:
* single column or list of columns
* `Not`: keep all except the listed columns
* `Between`: keep all columns between the given columns
* `Cols`: complex column selection rules (including function)

Going back to `bank2` data frame we notice that there is some strange `999` value in the last column. Let us investigate this.

In [ ]:
bank2

In [ ]:
describe(bank2)

In [ ]:
describe(bank2, :all)

(notice that the result is also a `DataFrame`, so you can work with it programatically in the future if you want)

We can inspect the contents of the column in more detail using a FreqTables.jl package

In [ ]:
freqtable(bank2, "pdays")

In [ ]:
proptable(bank2, "pdays")

Alternatively we could have used split-apply-combine strategy to get a similar result:

In [ ]:
combine(groupby(bank2, :pdays, sort=true), nrow, proprow)

It turns out that `999` means that the client was never contacted by a bank before

Assume we want to work only with customers who were contacted previously. We can subset `bank2` data frame.

In [ ]:
bank3 = subset(bank2, :pdays => x -> x .!= 999)

Here are some other options that do the same:

In [ ]:
subset(bank2, :pdays => ByRow(!=(999)))

In [ ]:
bank2[bank2.pdays .!= 999, :]

In [ ]:
bank2[Not(bank2.pdays .== 999), :]

In [ ]:
describe(bank3, mean => "mean", var => "var")

What is the distribution of `"age"`?

In [ ]:
histogram(bank3.age; label="age", normalize=true)
m = mean(bank3.age)
s = std(bank3.age)
plot!(10:100, pdf(Normal(m, s), 10:100); label="theoretical")


What is the distribution of `"campaign"` - "number of contacts performed during this campaign")?

In [ ]:
prop_campaign = combine(groupby(bank3, "campaign"), proprow)

In [ ]:
λ = mean(bank3.campaign) - 1.0 # note that the first contact is certain
prop_campaign.poisson = [pdf(Poisson(λ), k-1) for k in prop_campaign.campaign]
prop_campaign

In [ ]:
groupedbar(prop_campaign.campaign, [prop_campaign.proprow prop_campaign.poisson];
           xticks=1:13, bar_width=0.8, labels=["campaign" "theoretical"])

One might ask if the variables are correlated with `"y"` in `bank3` data frame. It is easy to check.

Recode variable `y` to be binary. Let us first make sure that it is binary indeed.

In [ ]:
combine(groupby(bank3, "y"), nrow)

OK - it is safe to recode it.

In [ ]:
bank3.y_bin = bank3.y .== "yes"

Always double check if you got what you wanted.

In [ ]:
freqtable(bank3, "y", "y_bin")

In [ ]:
combine(groupby(bank3, "y"), ["age", "campaign", "pdays"] .=> mean)

In [ ]:
combine(bank3, ["age", "campaign", "pdays"] .=> (x -> cor(x, bank3.y_bin)) .=> x -> x * "_cor")

In [ ]:
plot(groupedboxplot(bank3.y, bank3.age; xlabel="age", label=nothing),
     groupedboxplot(bank3.y, bank3.campaign; xlabel="campaign", label=nothing),
     groupedboxplot(bank3.y, bank3.pdays; xlabel="pdays", label=nothing))

In [ ]:
plot([groupedboxplot(bank3.y, bank3[:, col]; xlabel=col, label=nothing)
      for col in ["age", "campaign", "pdays"]]...)

## Handling missing data

Let us go back to our `bank2` data frame.

In [ ]:
bank2

We know that if `"pdays"` is `999` then it is a missing value. We can perform the conversion in several ways.

In [ ]:
bank2.pdays2 = replace(bank2.pdays, 999=>missing)

Remember that `missing` usses three valued logic. The `isequal` and `isless` functions do not pass missing, other standard operators return missing if they encounter missing.

In [ ]:
1 < missing

In [ ]:
isless(1, missing)

In [ ]:
isless(Inf, missing)

by the way, it is good to remember that `missing` is larger than any real value in `isless` comparison.

Let us see another example to see that it matters.

In [ ]:
subset(bank2, "pdays2" => x -> x .== 1, skipmissing=true)

or

In [ ]:
subset(bank2, "pdays2" => ByRow(isequal(1)))

In [ ]:
describe(bank2)

You have to change the type of the vector using the `allowmissing!` function.

In [ ]:
allowmissing!(bank2, "pdays")

Now you can assign `missing` to `"pdays"` column.

In [ ]:
replace!(bank2.pdays, 999=>missing)

In [ ]:
describe(bank2)

Often we want to get rid of missing data from a data frame. We can do this using the `dropmissing` function.

In [ ]:
bank4 = dropmissing(bank2)

Note that now columns automatically disallow missing values.

## Categorical variables

Sometimes you want to add to your columns the metadata that they are categorical.

In this section we discuss how you can do it and how this information is used by the `GLM` package as an application example.

Let us get back to our original `bank` data frame. We will want to build a model explaining `:y` using some other variables from the data set.

In [ ]:
bank

To estimate the model with `GLM` package needs to know that `:y` is binary. Let us change this.

In [ ]:
bank.y_bin = ifelse.(bank.y .== "yes", 1, 0)

In [ ]:
bank

In [ ]:
glm(@formula(y_bin ~ age + day_of_week + campaign), bank, Binomial())

This is not very nice as `day_of_week` is not properly ordered. Fortunately it is easy to change this.

In [ ]:
bank.day_of_week = categorical(bank.day_of_week)

In [ ]:
levels(bank.day_of_week)

In [ ]:
levels!(bank.day_of_week, ["mon", "tue", "wed", "thu", "fri"]);

Now the order of levels should be correct.

In [ ]:
glm(@formula(y_bin ~ age + day_of_week + campaign), bank, Binomial())

Let us have a look again at the `"campaign"` variable distribution.

In [ ]:
histogram(bank.campaign, label="campaign")

It is very skewed. We will cut it into several bins.

In [ ]:
bank.campaign_cat = cut(bank.campaign, 1:4; labels=["1", "2", "3", "4+"], extend=true)

In [ ]:
levels(bank.campaign_cat)

In [ ]:
proptable(bank.campaign_cat)

In [ ]:
glm(@formula(y_bin ~ age + day_of_week + campaign_cat), bank, Binomial())

Now let us investigate the `"age"` variable again.

In [ ]:
histogram(bank.age; label="age")

In [ ]:
bank.age_cut = cut(bank.age, 10)

In [ ]:
y_by_age = combine(groupby(bank, :age_cut), proprow, :y_bin => mean => :mean_y_bin)

In [ ]:
bar(y_by_age.age_cut, y_by_age.mean_y_bin; rotation=20, label=false)

It looks like our relationship is not linear. Let us add a quadratic term to the model.

In [ ]:
glm(@formula(y_bin ~ age + age^2 + day_of_week + campaign_cat), bank, Binomial())